In [ ]:
import JupyterNotebooksLib as slicernb
import slicer, qt, ctk, SampleData, time
from pathlib import Path

slicer.app.layoutManager().setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)
viewNodes = slicer.util.getNodesByClass("vtkMRMLAbstractViewNode")
for viewNode in viewNodes:
    viewNode.SetOrientationMarkerType(slicer.vtkMRMLAbstractViewNode.OrientationMarkerTypeAxes)

defaultMarkupsDisplayNode = slicer.vtkMRMLMarkupsDisplayNode()
defaultMarkupsDisplayNode.SetGlyphScale(2.5)
defaultMarkupsDisplayNode.SetOpacity(0.9)
defaultMarkupsDisplayNode.SetGlyphTypeFromString("Sphere3D")
defaultMarkupsDisplayNode.SetHandlesInteractive(True) 
defaultMarkupsDisplayNode.SetTranslationHandleVisibility(True)
defaultMarkupsDisplayNode.SetRotationHandleVisibility(True)
slicer.mrmlScene.AddDefaultNode(defaultMarkupsDisplayNode)
planeHI = slicer.vtkMRMLMarkupsPlaneDisplayNode()
planeHI.SetHandlesInteractive(True)
planeHI.SetTranslationHandleVisibility(True)
planeHI.SetRotationHandleVisibility(True)
planeHI.SetOpacity(0.9)
slicer.mrmlScene.AddDefaultNode(planeHI)

saveWindow = qt.QWidget()
saveLayout = qt.QVBoxLayout()
saveCheckBox = qt.QCheckBox('Segmentazione corretta')
textInfo = qt.QLabel('Cliccando sul tasto save verrà salvata la scena in formato .mrb e verranno esportatele coordinate dei punti dei piani')
saveButton = qt.QPushButton('Save')
saveDynamicSpacer = ctk.ctkDynamicSpacer()
saveWindow.setFixedSize(500, 300)
saveButton.setEnabled(False)
textInfo.setWordWrap(True)
saveLayout.addWidget(saveCheckBox)
saveLayout.addWidget(textInfo)
saveLayout.addWidget(saveDynamicSpacer)
saveLayout.addWidget(saveButton)   
saveDialog = qt.QMessageBox()
saveDialog.setModal(True) #Fino a quando non si chiude la finestra di dialogo non si può continuare   
exportDialog = qt.QFileDialog()
exportDialog.setFileMode(4)    
dirName = ''
def functionButtonSave():
    # Generate file name
    dirName = exportDialog.getExistingDirectory()  
    cont = 0
    volumes = slicer.mrmlScene.GetNodesByClass('vtkMRMLVolumeNode')
    for volume in volumes:
        volumeName = volume.GetName()
    fileName = volumeName + '-' + time.strftime('%Y-%m-%d')
    sceneSaveFilename = dirName + '/' +  volumeName + '-' + time.strftime('%Y-%m-%d') + '.mrb'
    if dirName != '':
        while cont == 0:            
            nameDialog = qt.QInputDialog()
            nameDialog.setWindowTitle('File Name')
            nameDialog.setLabelText("File Name:")
            nameDialog.setTextEchoMode(qt.QLineEdit().Normal)
            nameDialog.setTextValue('%s'%fileName)
            nameDialog.setOkButtonText('Save')
            if nameDialog.exec_() == qt.QDialog.Accepted:
                fileName = nameDialog.textValue()
                sceneSaveFilename = dirName + '/' + str(fileName) + ".mrb"
                if Path(sceneSaveFilename).is_file():
                    messageBoxSave = qt.QMessageBox()
                    messageBoxSave.setText("The file: '%s' already exists. \nDo you want to replace it?:" %fileName)
                    replaceButton = qt.QPushButton('Yes')
                    renameButton = qt.QPushButton('No, rename')
                    cancelButton = qt.QPushButton('Cancel')
                    messageBoxSave.addButton(replaceButton, 0)
                    messageBoxSave.addButton(renameButton, 0)
                    messageBoxSave.addButton(cancelButton, 0)
                    mb = messageBoxSave.exec()
                    if mb == 0:
                        cont = 1
                        exportPointsFunction()
                        saveTableFunction(dirName, fileName)
                        if slicer.util.saveScene(sceneSaveFilename):
                            saveDialog.setText('Scene saved')
                            saveDialog.show()
                        else:
                            saveDialog.setText('Scene saving failed')
                            saveDialog.show()
                    elif mb == 1:
                        cont = 0
                    elif mb == 2:
                        cont = 1
                        nameDialog.close()
                else:
                    cont = 1
                    saveTableFunction(dirName, fileName)
                    if slicer.util.saveScene(sceneSaveFilename):
                        saveDialog.setText('Scene saved')
                        saveDialog.show()
                    else:
                        saveDialog.setText('Scene saving failed')
                        saveDialog.show()
                saveWindow.close()
            else:
                cont = 1
                saveDialog.setText('Scene saving failed')
                saveDialog.show()
                nameDialog.close()
                saveWindow.close()
def functionButton(bool):
    if saveButton.enabled == False:
        saveButton.setEnabled(True)
        saveButton.connect('clicked()', functionButtonSave)
    else:
        saveButton.setEnabled(False)
exportDialogMessage = qt.QMessageBox()
exportDialogMessage.setModal(True)
def saveTableFunction(dirName, fileName):
    tables = slicer.mrmlScene.GetNodesByClass('vtkMRMLTableNode')
    for table in tables:
        if (table.GetColumnName(1)) == 'r': 
            filenameRAS = dirName + '/' + fileName + '-' + table.GetName() + '-RAS.txt'
            slicer.util.saveNode(table, filenameRAS)
        elif (table.GetColumnName(1)) == 'l':
            filenameLPS = dirName + '/' + fileName +  '-' + table.GetName() + '-LPS.txt'
            slicer.util.saveNode(table, filenameLPS)
def showSimpleUserInterface(simpleView):

    # Do not store these display settings permanently
    settings = qt.QSettings()
    settings.setValue('MainWindow/RestoreGeometry', not simpleView)

    for toolbar in slicer.util.mainWindow().findChildren('QToolBar'):
        toolbar.setVisible(False)

    slicer.util.setDataProbeVisible(False) # Menù Data probe
    slicer.util.setApplicationLogoVisible(False) # Logo slicer
    
    slicer.qMRMLMarkupsDisplayNodeWidget().setPropertiesLabelVisibility(True)
          
    layoutManager = slicer.app.layoutManager()
    for sliceViewName in layoutManager.sliceViewNames():
        sliceWidget = layoutManager.sliceWidget(sliceViewName)
        sliceControlWidget = sliceWidget.sliceController()
        sliceControlWidget.setSliceVisible(False) #Piani nella vista 3D
        pinButton = sliceControlWidget.children()[1].children()[1]
        if simpleView:
            pinButton.hide()
        else:
            pinButton.show()
                
def saveFunction():
    #Blocco tutti i nodi prima di procede al salvataggio
    nodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLMarkupsNode')
    for node in nodes:
        for i in range(0,node.GetNumberOfControlPoints()):
            node.SetNthControlPointLocked(i,True)
    #Controllo che l'utente abbia esportato le tabelle con i punti dei piani
    tables = slicer.mrmlScene.GetNodesByClass('vtkMRMLTableNode')
    if tables.GetNumberOfItems() != 0:
        saveCheckBox.connect('clicked(bool)', functionButton)
        saveWindow.setLayout(saveLayout)
        saveWindow.show()  
    else:
        msg = qt.QMessageBox()
        msg.setText('Per procede al salvataggio occorre esportare le tabelle dei punti del/i piano/i')
        msg.exec()       

def createSimpleDataBrowser():    
    mainWindow = slicer.util.mainWindow()
    dockWidget = qt.QDockWidget("Data browser", mainWindow)
    dockWidget.setFeatures(qt.QDockWidget.DockWidgetClosable) # Permettere il riposizionamento della finestra: + qt.QDockWidget.DockWidgetMovable)
    dockWidget.setMinimumWidth(10)
    dockWidget.setMaximumWidth(570)
    mainFrame = qt.QFrame(dockWidget)    
    mainFrameLayout = qt.QVBoxLayout(mainFrame) 
             
    collapsibleButton_Data = ctk.ctkCollapsibleButton()
    collapsibleButton_Data.text = 'Data'
    collapsibleButton_Data_Layout = qt.QFormLayout(collapsibleButton_Data)   
    loadDataButton = qt.QPushButton('Load Data')
    collapsibleButton_Data_Layout.addRow(loadDataButton)
    loadDataButton.connect('clicked()', slicer.util.openAddDataDialog)
    saveDataButton = qt.QPushButton('Save Data')
    collapsibleButton_Data_Layout.addRow(saveDataButton)
    saveDataButton.connect('clicked()', saveFunction)  
    collapsibleButton_Data.collapsed = True
    
    collapsibleButton_Layout = ctk.ctkCollapsibleButton()
    collapsibleButton_Layout.text = 'Layout'
    collapsibleButton_Layout_Layout = qt.QHBoxLayout(collapsibleButton_Layout)
    displayPresets = (
        (slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView, "LayoutFourUpView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpRedSliceView, "LayoutOneUpRedSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpYellowSliceView, "LayoutOneUpYellowSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpGreenSliceView, "LayoutOneUpGreenSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUp3DView, "LayoutOneUp3DView.png"),
        )
    for (layoutId, iconName) in displayPresets:
        switchLayoutButton = qt.QPushButton("")
        switchLayoutButton.setIcon(qt.QIcon(":Icons/"+iconName))
        collapsibleButton_Layout_Layout.addWidget(switchLayoutButton, 0, 0)
        switchLayoutButton.connect('clicked()', lambda layoutId=layoutId: slicer.app.layoutManager().setLayout(layoutId))
    collapsibleButton_Layout.collapsed = True
    
    collapsibleButton_Markups = ctk.ctkCollapsibleButton()
    collapsibleButton_Markups.text = 'Markups'    
    collapsibleButton_Markups_Form = qt.QFormLayout(collapsibleButton_Markups)
    collapsibleButton_Markups_Layout = qt.QHBoxLayout()    
    markups = slicer.modules.markups.createNewWidgetRepresentation()
    markups.enter()    
    slicer.util.findChild(markups, "createMarkupsGroupBox").hide()
    slicer.util.findChild(markups, "displayCollapsibleButton").hide()
    slicer.util.findChild(markups, "measurementsCollapsibleButton").hide()
    slicer.util.findChild(markups, "advancedCollapsibleButton").hide()
    groupBoxCreateMarkups = qt.QGroupBox()
    layoutCreateMarkups = qt.QHBoxLayout()
    groupBoxCreateMarkups.setLayout(layoutCreateMarkups)
    fiducialPushButton = slicer.util.findChild(markups, "CreateFiducialPushButton")
    linePushButton = slicer.util.findChild(markups, "CreateLinePushButton")
    anglePushButton = slicer.util.findChild(markups, "CreateAnglePushButton")
    planePushButton = slicer.util.findChild(markups, "CreatePlanePushButton")
    layoutCreateMarkups.addWidget(fiducialPushButton)
    layoutCreateMarkups.addWidget(linePushButton)
    layoutCreateMarkups.addWidget(anglePushButton)
    layoutCreateMarkups.addWidget(planePushButton)    
    collapsibleButton_Markups_Layout.addWidget(markups)
    collapsibleButton_Markups_Form.addRow(groupBoxCreateMarkups)
    collapsibleButton_Markups_Form.addRow(collapsibleButton_Markups_Layout)    
    collapsibleButton_Markups.collapsed = True

    collapsibleButton_Texts = ctk.ctkCollapsibleButton()
    collapsibleButton_Texts.text = 'Texts'    
    collapsibleButton_Texts_Form = qt.QFormLayout(collapsibleButton_Texts)
    collapsibleButton_Texts_Layout = qt.QHBoxLayout()    
    text = slicer.modules.texts.createNewWidgetRepresentation()    
    slicer.util.findChild(text, "CollapsibleButton").hide()    
    collapsibleButton_Texts_Layout.addWidget(text)
    collapsibleButton_Texts_Form.addRow(collapsibleButton_Texts_Layout)
    collapsibleButton_Texts.collapsed = True
        
    area = qt.QScrollArea()
    area.setWidgetResizable(True)
    widget = qt.QWidget()
    area.setWidget(widget)
    layoutWidget = qt.QVBoxLayout()
    widget.setLayout(layoutWidget)        
    layoutWidget.addWidget(collapsibleButton_Data)
    layoutWidget.addWidget(collapsibleButton_Layout)
    layoutWidget.addWidget(collapsibleButton_Markups)
    layoutWidget.addWidget(collapsibleButton_Texts)
    layoutWidget.addStretch(1)
    mainFrameLayout.addWidget(area)

    dataTreeWidget = slicer.qMRMLSubjectHierarchyTreeView(mainFrame)
    dataTreeWidget.setMRMLScene(slicer.mrmlScene)
    dataTreeWidget.setColumnHidden(dataTreeWidget.model().idColumn, True)
    dataTreeWidget.setColumnHidden(dataTreeWidget.model().transformColumn, True)
    dataTreeWidget.setMaximumHeight(200)
    mainFrameLayout.addWidget(dataTreeWidget)
    dockWidget.setWidget(mainFrame)
    mainWindow.addDockWidget(qt.Qt.LeftDockWidgetArea, dockWidget)
        
showSimpleUserInterface(False)
createSimpleDataBrowser()
#slicernb.AppWindow()

from ipywidgets import HTML
HTML(f"""<a href="{slicernb.AppWindow.defaultDesktopUrl()}" target="_blank">
<b>Click here</b> to open application window in a new browser tab.</a>""")